# 第11章 関数型プログラムを設計する

## 11.1 まず動かす、正しく動かす、高速に動かす

節タイトルはKent Beckの引用らしい。

**要件: ポップカルチャー旅行ガイド**

1. このアプリケーションはString型の値を1つだけ受け取る。この値は、ユーザーが訪れたいと考えていて、  
   旅行ガイドを必要としている観光名所の検索語である。
2. このアプリケーションは観光名所、その説明(説明がある場合)、その地理的位置で検索を行う必要がある。  
   人口の多い場所を優先する。
3. このアプリケーションは場所を使って次のことを行う:
   - この場所出身のアーティストを検索し、ソーシャルメディアのフォロワー数の順に並べる
   - この場所を舞台にした映画を検索し、興行収入の順に並べる
4. 特定の観光名所とその地にゆかりがあるアーティストと映画から「ポップカルチャー旅行ガイド」を作成し、ユーザーに返す。  
   ガイドが他にもある場合は、「スコア」が最も高いものを返す必要がある。  
   スコアは次のように計算する:
   - 説明は30ポイント
   - アーティストまたは映画ごとに10ポイント
   - フォロワー10万人につき1ポイント(全アーティストを合わせて。最大15ポイント)
   - 興行収入1,000万ごとに1ポイント(すべての映画を合わせて。最大15ポイント)
5. 将来的には、さらに多くのポップカルチャーテーマ(ビデオゲームなど)をサポートする予定。

```scala
// 実行例
travelGuideProgram("Bridge of Sighs").unsafeRunSync()
// →「溜息橋はベネチアの運河に架かる橋。当地を訪れる前に、Talcoの曲を聴き、
// 「スパイダーマン：ファー・フロム・ホーム」や「カジノ・ロワイヤル」など、
// ベネチアを舞台とする映画を観てはいかが。」
```

## 11.2 イミュータブルな値を使ってモデル化する

要件が明確に指定されている場合、イミュータブルな値(直積型と直和型、またはADT)を使ったモデル化は非常に簡単である。

## 11.3 ビジネスドメインのモデル化と関数型プログラミング

In [1]:
object model {
  opaque type LocationId = String
  object LocationId{
    def apply(value: String): LocationId        = value
    extension (a: LocationId) def value: String = a
  }

  case class Location(id: LocationId, name: String, population: Int)
  case class Attraction(name: String, description: Option[String], location: Location)

  enum PopCultureSubject {
    case Artist(name: String, followers: Int)
    case Movie(name: String, boxOffice: Int)
  }

  case class TravelGuide(attraction: Attraction, subjects: List[PopCultureSubject])
}

import model._, model.PopCultureSubject._


defined object model
import model._, model.PopCultureSubject._



> なお、ここではアプリケーション全体のすべてのレイヤ(層)で同じモデルを使うことにした。  
> しかし、もっと大規模なアプリケーションでは、それぞれの概念をその概念が使われるコンテキストに応じて表現するほうが  
> 通常は賢明である(DDDと**境界付きコンテキスト**の概念を確認しておこう)。この場合、  
> Artistモデルが意味を持つのはガイドのスコアを計算するコンテキストだけであると言ってよいだろう。  
> なぜなら、followersが必要になるのはそのコンテキストだからだ。このクラスは他のコンテキストでも使われるが、  
> 最終的なユーザープレゼンテーション(UIなど)では、もう少し単純なアーティストモデルのほうがよいことが想像できる。

```scala
case class ArtistToListenTo(name: String)
```

## 11.4 データアクセスをモデル化する

In [2]:
import $ivy.`org.typelevel::cats-effect:3.2.9`
import $ivy.`co.fs2::fs2-core:3.1.2`

import cats.effect._

// "…この場所出身のアーティストを検索…"
def findArtistsFromLocation(locationId: LocationId, limit: Int): IO[List[Artist]] = ???

// "…この場所を舞台とした映画を検索…"
def findMoviesAboutLocation(locationId: LocationId, limit: Int): IO[List[Movie]] = ???

// "このアプリケーションは特定の観光名所を検索する必要がある…"
def findAttractions(name: String, ordering: AttractionOrdering, limit: Int): IO[List[Attraction]] = ???

enum AttractionOrdering {
  case ByName
  case ByLocationPopulation
}
import AttractionOrdering._


import $ivy.$                                 

import $ivy.$                       


import cats.effect._

// "…この場所出身のアーティストを検索…"

defined function findArtistsFromLocation
defined function findMoviesAboutLocation
defined function findAttractions
defined class AttractionOrdering
import AttractionOrdering._



## 11.5 BoF

> 通常は一緒に使われる一連の関数がある、またはそれらの関数の実装に共通する式がたくさんある場合は、  
> それらの関数をより大きな型――いわゆる**BoF**(bag of functions)にまとめたくなるかもしれない。

In [3]:
trait DataAccess {
  def findArtistsFromLocation(locationId: LocationId, limit: Int): IO[List[Artist]]
  def findMoviesAboutLocation(locationId: LocationId, limit: Int): IO[List[Movie]]
  def findAttractions(name: String, ordering: AttractionOrdering, limit: Int): IO[List[Attraction]]
}

// trait...interfaceみたいなやつ


defined trait DataAccess

直積型とインターフェイスのどちらのバージョンを選択肢たとしても、メリットは同じである。

- 3つの関数の値を個別に渡すのではなく、データアクセス機能を必要とする**関数に1つの値を渡す**。  
  データアクセスを要求する関数の数が多い場合、この利点はさらに重要となる。そのような場合、  
  こうした肥大化したシグネチャは非常に反復的に見えるため、コードが乱雑に見える。
- 共通の式を使って**すべての関数を一度に実装する**。

> 欠点も少なくとも1つある。多くの関数に適した「共通のアイデンティティ」を見つけ出すのが難しいことだ。  
> この場合は「データアクセス」アイデンティティを共有する関数が3つあるため、それほど難しくない。  
> しかし、大量の関数が含まれた大規模なプログラムでは、それではうまくいかないだろう。

## 11.6 純粋関数としてのビジネスロジック

最も単純なバージョンであるIO型の値を返す関数を実装する。

In [4]:
def travelGuide(data: DataAccess, attractionName: String): IO[Option[TravelGuide]] = {
  for {
    attractions <- data.findAttractions(attractionName, ByLocationPopulation, 1)
    guide       <- attractions.headOption match {
                    case None => IO.pure(None)
                    case Some(attraction) =>
                      for {
                        artists <- data.findArtistsFromLocation(attraction.location.id, 2)
                        movies  <- data.findMoviesAboutLocation(attraction.location.id, 2)
                      } yield Some(TravelGuide(attraction, artists.appendedAll(movies)))
                  }
  } yield guide
}


defined function travelGuide

## 11.7 現実のデータアクセスでの関心の分離

travelGuide関数はDataAccess型の値を使う。この値は、travelGuideが知る必要があるすべての情報を、  
内部の詳細をいっさい明かすことなく提供する。  
travelGuideを呼び出すには、最初にそのようなDataAccessを作成しなければならない。

Wikidataでは、Wikidataが公開しているデータを、SPARQLクエリ言語を使って検索できる。

## 11.8 命令型ライブラリとIOを使ってAPIを統合する

findAttractionsから実装していく。  
この関数を実装する前に、次の3つの問題を解決する必要がある。

1. どのようなSPARQLクエリで実装するのか
2. Wikidata APIに接続してクエリを実行するにはどうすればよいか
3. 返されたレスポンスをList[Attraction]型の値に変換するにはどうすればよいか

### 1. SPARQLクエリを作成する

特定の名前を含んでいて、名前または人口で並べ替えることができる観光名所を検索したい。  
[Wikidata Query Service](https://query.wikidata.org)にアクセスして、そのようなクエリを実装する。

### 2. Wikidataサービスに接続し、検索を行う

> Apache Jeneは、Wikidataサーバーに接続してクエリを実行するための、  
> Javaベースの命令形のクライアントライブラリであり、「セマンティックWebアプリと  
> リンクトデータアプリを構築するための、オープンソースのフリーのJavaフレームワーク」  
> である。

命令型のAPIでも、関数型プログラムに組み込むことができる。

### 3. クエリの結果を抽出して解析する

コードは本文を参照。

命令型のライブラリのコードをIO型の値でラップし、関数型プログラムで安全に使えるようにする必要がある。

## 11.9 設計に従う

Wikidataでクエリを実行する方法がわかったので関数型プログラムに組み込んでいく。  
**DataAccess内で定義された3つの関数を実装する必要がある。それらは内部でApache Jenaを使う。**

## 11.10 入力アクションをIO型の値として実装する

In [4]:
import $ivy.`org.apache.jena:jena-arq:3.17.0`
import $ivy.`org.apache.jena:jena-core:3.17.0`
import $ivy.`org.apache.jena:jena-iri:3.17.0`
import $ivy.`org.apache.jena:jena-shaded-guava:3.17.0`
import $ivy.`org.apache.jena:jena-sparql-expr-arq:3.17.0`   // 依存関係を解消できない…
import $ivy.`org.apache.jena:jena-tdb:3.17.0`
import $ivy.`org.apache.jena:jena-tdb2:3.17.0`
import $ivy.`org.apache.jena:apache-jena-libs:3.17.0`
import org.apache.jena.query._
import org.apache.jena.rdfconnection._

val getConnection: IO[RDFConnection] = IO.delay(
  RDFConnectionRemote.create
    .destination("https://query.wikidata.org/")
    .queryEndpoint("sparql").build
)


Downloaded https://repo1.maven.org/maven2/org/apache/jena/jena-sparql-expr-arq/3.17.0/jena-sparql-expr-arq-3.17.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/jena/jena-sparql-expr-arq/3.17.0/jena-sparql-expr-arq-3.17.0.pom.sha1
Downloaded https://jitpack.io/org/apache/jena/jena-sparql-expr-arq/3.17.0/jena-sparql-expr-arq-3.17.0.pom
Downloaded https://jitpack.io/org/apache/jena/jena-sparql-expr-arq/3.17.0/jena-sparql-expr-arq-3.17.0.pom.sha1
Failed to resolve ivy dependencies:Error downloading org.apache.jena:jena-sparql-expr-arq:3.17.0
  not found: C:\Users\nerik\.ivy2\local\org.apache.jena\jena-sparql-expr-arq\3.17.0\ivys\ivy.xml
  not found: https://repo1.maven.org/maven2/org/apache/jena/jena-sparql-expr-arq/3.17.0/jena-sparql-expr-arq-3.17.0.pom
  not found: https://jitpack.io/org/apache/jena/jena-sparql-expr-arq/3.17.0/jena-sparql-expr-arq-3.17.0.pom